In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
directory = "Data"
pairs = [['image1a.jpeg','image1b.jpeg'],['image2a.jpeg','image2b.jpeg'],['image3a.jpeg','image3b.jpeg'],['image4a.jpeg','image4b.jpeg','image4c.jpeg'],
         ['image5a.jpeg','image5b.jpeg'], ['image6a.jpeg','image6b.jpeg'],['image7a.jpeg','image7b.jpeg']]

In [3]:
def getSIFTKpsAndDecriptors(image):
    SIFTMatcher = cv2.SIFT_create()
    (keypoints, descriptors) = SIFTMatcher.detectAndCompute(image,None)
    #keypoints = np.float32([keypoints.pt for keypoint in keypoints])
    return keypoints, descriptors

In [4]:
def getMatches(descriptors1, descriptors2):
    bruteForceMatcher = cv2.BFMatcher()
    matches = bruteForceMatcher.knnMatch(descriptors1,descriptors2,2)
    return matches

In [5]:
def getDavidLoweRatioMatches(matches,davidLoweRatio):
    DLmatches =[]
    for match in matches:
        if(match[0].distance/match[1].distance < davidLoweRatio):
                DLmatches.append(match[0])
    return DLmatches

In [6]:
def getDistancesArray(matches):
    distanceArray = []
    for match in matches:
        distanceArray.append(match.distance)
    return np.array(distanceArray)

In [7]:
def getInterQuartileRange(distances):
    firstQuartile = np.percentile(distances,25)
    thirdQuartile = np.percentile(distances,75)
    interQuartileRange = thirdQuartile - firstQuartile
    return interQuartileRange

In [8]:
def filterMatchesUsingIQR(interQuartileRange, DLmatches):
    IQRMatches = []
    for match in DLmatches:
        if (match.distance < interQuartileRange):
            IQRMatches.append(match)
    return IQRMatches

In [12]:
def run(leftPairDirectory,rightPairDirectory):
    leftPair = cv2.imread(os.path.join(directory,leftPairDirectory),cv2.IMREAD_GRAYSCALE)
    rightPair = cv2.imread(os.path.join(directory,rightPairDirectory),cv2.IMREAD_GRAYSCALE)
    leftPair = cv2.resize(leftPair,(500,500))
    rightPair = cv2.resize(rightPair,(500,500))
    keyPointsLeft, descriptorsLeft = getSIFTKpsAndDecriptors(leftPair)
    keyPointsRight, descriptorsRight = getSIFTKpsAndDecriptors(rightPair)
    matches = getMatches(descriptorsLeft, descriptorsRight)
    DLmatches = getDavidLoweRatioMatches(matches,0.8)
    distances = getDistancesArray(DLmatches)
    IQR = getInterQuartileRange(distances)
    IQRMatches = filterMatchesUsingIQR(IQR, DLmatches)
    vis = cv2.drawMatches(leftPair,keyPointsLeft,rightPair,keyPointsRight,IQRMatches,outImg = None,matchesThickness=2)
    cv2.imshow("img",vis)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
run(pairs[0][0],pairs[0][1])